In [1]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import math
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

import psycopg2
import pandas.io.sql as psql
from tqdm.autonotebook import tqdm

In [4]:
###Prepare Data#####
data = pd.read_csv ('/Users/pinnytu/Downloads/data_MarketBasket.csv')
data

,Unnamed: 0,customer_id,cleaned_brandname
0,0,CUSTOMER1,ORIGINS
1,1,CUSTOMER1,BIOTHERM
2,2,CUSTOMER1,LANEIGE
3,3,CUSTOMER1,KIEHLS
4,4,CUSTOMER10,MAC
...,...,...,...
1217669,1217669,CUSTOMER999997,CHANEL
1217670,1217670,CUSTOMER999997,MAC
1217671,1217671,CUSTOMER999997,LAURA MERCIER
1217672,1217672,CUSTOMER999997,SISLEY


In [6]:
#prep data
def asso_rule_fn(data,column_id,interested_column):
    asso_data=[]
    temp=[]
    start_data=data.loc[0,column_id]
    for i in range(len(data)):
        current_data=data.loc[i,column_id]
        if start_data==current_data:
            temp.append(data.loc[i,interested_column])
        else:
            start_data=current_data
            asso_data.append(temp)
            temp=[]
            temp.append(data.loc[i,interested_column])
    return asso_data

In [7]:
data_for_asso = asso_rule_fn(data,'customer_id','cleaned_brandname')

In [8]:
pd.DataFrame(data_for_asso)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,ORIGINS,BIOTHERM,LANEIGE,KIEHLS,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,MAC,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,SHISEIDO,MAC,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,CHRISTIAN DIOR,THREE,KIEHLS,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,CHANEL,MAC,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672599,CHRISTIAN DIOR,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
672600,LANEIGE,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
672601,COVERMARK,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
672602,COVERMARK,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
te = TransactionEncoder()
te_ary = te.fit(data_for_asso).transform(data_for_asso)

In [10]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [11]:
df

,ACSEINE,AESOP,ANNA SUI,BENEFIT,BIOTHERM,BOBBI BROWN,BODY SHOP,BURBERRY,BVLGARI,CALVIN KLEIN,...,SUQQU,THREE,TOM FORD BEAUTY,ULTIMA II,URBAN DECAY,VALMONT,VERSACE,VIKTOR & ROLF,WHOO,YVES SAINT LAURENT
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672599,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672600,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672601,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672602,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
df.to_pickle('data_df.pickle')

In [13]:
df = pd.read_pickle('data_df.pickle')

In [14]:
data['customer_id']

0               CUSTOMER1
1               CUSTOMER1
2               CUSTOMER1
3               CUSTOMER1
4              CUSTOMER10
                ...      
1217669    CUSTOMER999997
1217670    CUSTOMER999997
1217671    CUSTOMER999997
1217672    CUSTOMER999997
1217673    CUSTOMER999997
Name: customer_id, Length: 1217674, dtype: object

In [15]:
used = set()
mylist = data['customer_id']
unique = [x for x in mylist if x not in used and (used.add(x) or True)]

In [17]:
data_prep = pd.concat([pd.DataFrame(unique),df], axis = 1)

In [18]:
data_prep.to_csv('data_prep.csv')

In [19]:
data_prep

,0,ACSEINE,AESOP,ANNA SUI,BENEFIT,BIOTHERM,BOBBI BROWN,BODY SHOP,BURBERRY,BVLGARI,...,SUQQU,THREE,TOM FORD BEAUTY,ULTIMA II,URBAN DECAY,VALMONT,VERSACE,VIKTOR & ROLF,WHOO,YVES SAINT LAURENT
0,CUSTOMER1,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,CUSTOMER10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,CUSTOMER100,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,CUSTOMER1000,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,CUSTOMER10000,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672600,CUSTOMER999990,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672601,CUSTOMER999991,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672602,CUSTOMER999992,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
672603,CUSTOMER999993,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Brands purchased in same basket

In [20]:
frequent_itemsets = apriori(df, min_support=0.0001, use_colnames=True, low_memory = True, max_len = 2)

In [21]:
frequent_itemsets

,support,itemsets
0,0.000874,(ACSEINE)
1,0.009677,(AESOP)
2,0.000299,(ANNA SUI)
3,0.019833,(BENEFIT)
4,0.043135,(BIOTHERM)
...,...,...
1266,0.000410,"(TOM FORD BEAUTY, WHOO)"
1267,0.002507,"(YVES SAINT LAURENT, TOM FORD BEAUTY)"
1268,0.000596,"(YVES SAINT LAURENT, URBAN DECAY)"
1269,0.000546,"(VERSACE, YVES SAINT LAURENT)"


In [24]:
asso_rule=association_rules(frequent_itemsets,metric = 'lift')

In [25]:
asso_rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ACSEINE),(BOBBI BROWN),0.000874,0.115468,0.000169,0.193878,1.679064,0.000069,1.097268
1,(BOBBI BROWN),(ACSEINE),0.115468,0.000874,0.000169,0.001468,1.679064,0.000069,1.000595
2,(ACSEINE),(CHANEL),0.000874,0.118093,0.000153,0.175170,1.483320,0.000050,1.069198
3,(CHANEL),(ACSEINE),0.118093,0.000874,0.000153,0.001297,1.483320,0.000050,1.000423
4,(ACSEINE),(CHRISTIAN DIOR),0.000874,0.142662,0.000140,0.159864,1.120579,0.000015,1.020475
...,...,...,...,...,...,...,...,...,...
2139,(URBAN DECAY),(YVES SAINT LAURENT),0.004862,0.043642,0.000596,0.122630,2.809886,0.000384,1.090028
2140,(VERSACE),(YVES SAINT LAURENT),0.009851,0.043642,0.000546,0.055388,1.269132,0.000116,1.012434
2141,(YVES SAINT LAURENT),(VERSACE),0.043642,0.009851,0.000546,0.012503,1.269132,0.000116,1.002685
2142,(YVES SAINT LAURENT),(WHOO),0.043642,0.005158,0.000776,0.017783,3.447929,0.000551,1.012854


In [26]:
asso_rule.to_csv('Result_Beauty_LaMer.csv')